<a href="https://colab.research.google.com/github/anirbansen3027/feature-engineering/blob/master/missing_values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Missing values**

Missing data, or Missing values, occur when no data / no value is stored for a certain observation within a variable.

Missing data are a common occurrence both in data science competitions and in data in business settings, and can have a significant effect on the conclusions that can be drawn from the data. Incomplete data is an unavoidable problem in dealing with most data sources.

**Missing Data Mechanisms**

There are 3 mechanisms that lead to missing data, 2 of them involve missing data randomly or almost-randomly, and the third one involves a systematic loss of data.

1.Missing Completely at Random, MCAR: A variable is missing completely at random (MCAR) if the probability of being missing is the same for all the observations.

2.Missing at Random, MAR: The probability an observation being missing depends only on available information (other variables in the dataset).

3.Missing Not at Random, MNAR: Missing of values is not at random (MNAR) if their being missing depends on information not recorded in the dataset.

**Datasets:**


1.   Predicting Survival on the Titanic: understanding society behaviour and beliefs:
https://www.kaggle.com/c/titanic
2.  Peer to peer lending: Finance:
https://www.kaggle.com/wendykan/lending-club-loan-data



In [0]:
#getting the libraries
import pandas as pd
import numpy as np
from numpy import nan
import matplotlib.pyplot as plt

In [32]:
#Colab's file access feature
from google.colab import files
#retrieve uploaded file
uploaded = files.upload()
#move kaggle.json into the file where the api expects it to be there
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [3]:
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 16.2MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 23.6MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 1.92MB/s]


In [4]:
!ls

gender_submission.csv  sample_data  test.csv  train.csv


In [5]:
data = pd.read_csv('train.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


**1.Missing data Not At Random (MNAR): Systematic missing values:**
In this dataset, both the missing values of the variables Cabin and Age, were introduced systematically. For many of the people who did not survive, the age they had or the cabin they were staying in, could not be established. The people who survived could be asked for that information.

In [6]:
# you can determine the total number of missing values using
# the isnull method plus the sum method on the dataframe
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
# alternatively, you can call the mean method after isnull
# to visualise the percentage of the dataset that 
# contains missing values for each variable

data.isnull().mean()

PassengerId    0.000000
Survived       0.000000
Pclass         0.000000
Name           0.000000
Sex            0.000000
Age            0.198653
SibSp          0.000000
Parch          0.000000
Ticket         0.000000
Fare           0.000000
Cabin          0.771044
Embarked       0.002245
dtype: float64

We can see that there are missing data in the variables Age, Cabin (in which the passenger was travelling) and Embarked, which is the port from which the passenger got into the Titanic.

**Missing data Not At Random (MNAR): Systematic missing values**

In this dataset, both the missing values of the variables Cabin and Age, were introduced systematically. For many of the people who did not survive, the age they had or the cabin they were staying in, could not be established. The people who survived could be asked for that information.

Can we infer this by looking at the data?

In a situation like this, we could expect a greater number of missing values for people who did not survive.

In [8]:
# we create a dummy variable that indicates whether the value
# of the variable cabin is missing
data['cabin_null'] = np.where(data.Cabin.isnull(),1,0)

# find percentage of null values
data.cabin_null.mean()

0.7710437710437711

In [10]:
# and then we evaluate the mean of the missing values in
# cabin for the people who survived vs the non-survivors.

# group data by Survived vs Non-Survived
# and find nulls for cabin
data.groupby(['Survived'])['cabin_null'].mean()

Survived
0    0.876138
1    0.602339
Name: cabin_null, dtype: float64

We observe that the percentage of missing values is higher for people who did not survive (0.87), respect to people that survived (0.60). This finding is aligned with our hypothesis that the data is missing because after the people died, the information could not be retrieved.



In [11]:
# we repeat the exercise for the variable age:
# First we create a dummy variable that indicates
# whether the value of the variable Age is missing
data['age_null'] = np.where(data.Age.isnull(),1,0)

# and then look at the mean in the different survival groups:
# there are more NaN for the people who did not survive
data.groupby(['Survived'])['age_null'].mean()

Survived
0    0.227687
1    0.152047
Name: age_null, dtype: float64

Again, we observe an increase in missing data for the people who did not survive the tragedy. The analysis therefore suggests:

There is a systematic loss of data: people who did not survive tend to have more information missing. Presumably, the method chosen to gather the information, contributes to the generation of these missing data.

**2.Missing data Completely At Random (MCAR):**

In the titanic dataset, there were also missing values for the variable Embarked, let's have a look.

In [16]:
data[data.Embarked.isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,cabin_null,age_null
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN,0,0
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN,0,0


A priori, there does not seem to be an indication that the missing information in the variable Embarked is depending on any other variable, and the fact that these women survived, means that they could have been asked for this information.

Very likely this missingness was generated at the time of building the dataset and therefore we could assume that it is completely random. We can assume that the probability of data being missing for these 2 women is the same as the probability for this variable to be missing for any other person. Of course this will be hard, if possible at all, to prove.

**3.Missing data At Random (MAR):**
For this example, I will use the Lending Club loan book. I will look specifically at the variables employer name (emp_title) and years in employment (emp_length), declared by the borrowers at the time of applying for a loan. The former refers to the name of the company for which the borrower works, the second one to how many years the borrower has worked for named company.

Here I will show an example, in which a data point missing in one variable (emp_title) depends on the value entered on the other variable (emp_lenght).

In [33]:
!kaggle datasets download -d wendykan/lending-club-loan-data
!ls
!unzip lending-club-loan-data.zip

lending-club-loan-data.zip: Skipping, found more recently modified local copy (use --force to force download)
database.sqlite        LCDataDictionary.xlsx	   loan.csv	test.csv
gender_submission.csv  lending-club-loan-data.zip  sample_data	train.csv
Archive:  lending-club-loan-data.zip
replace LCDataDictionary.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace database.sqlite? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace loan.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [34]:
data=pd.read_csv('loan.csv', usecols=['emp_title','emp_length'])
data.head()

,emp_title,emp_length
0,NaN,10+ years
1,Ryder,< 1 year
2,NaN,10+ years
3,AIR RESOURCES BOARD,10+ years
4,University Medical Group,1 year


In [39]:
# let's inspect the variable emp_length
data.emp_length.unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', nan],
      dtype=object)

In [35]:
# let's look at the percentage of borrowers within
# each label / category of the emp_length variable

data.emp_length.value_counts() / len(data)

10+ years    0.328573
2 years      0.088880
< 1 year     0.079566
3 years      0.078913
1 year       0.064341
5 years      0.062774
4 years      0.059196
7 years      0.050254
8 years      0.049534
6 years      0.048401
9 years      0.039055
Name: emp_length, dtype: float64

In [42]:
# the variable emp_length has many categories. I will summarise it
# into 3 for simplicity:'0-10 years' or '10+ years' or 'n/a'
# let's build a dictionary to re-map emp_length to just 3 categories:
length_dict = {k:'0-10' for k in data.emp_length.unique()}
length_dict[nan] = 'nan'
length_dict['10+ years'] = '10+ years'
length_dict


{'1 year': '0-10',
 '10+ years': '10+ years',
 '2 years': '0-10',
 '3 years': '0-10',
 '4 years': '0-10',
 '5 years': '0-10',
 '6 years': '0-10',
 '7 years': '0-10',
 '8 years': '0-10',
 '9 years': '0-10',
 '< 1 year': '0-10',
 nan: 'nan'}

In [45]:
# let's re-map the emp_length
data['emp_length_redefined'] = data.emp_length.map(length_dict)
data.head()

,emp_title,emp_length,emp_length_redefined
0,NaN,10+ years,10+ years
1,Ryder,< 1 year,0-10
2,NaN,10+ years,10+ years
3,AIR RESOURCES BOARD,10+ years,10+ years
4,University Medical Group,1 year,0-10


In [53]:
# number of borrowers for whom employer name is missing
value = len(data[data.emp_title.isnull()])
value
data[data.emp_title.isnull()].groupby('emp_length_redefined')['emp_length_redefined'].count() / value

emp_length_redefined
0-10         0.082507
10+ years    0.054020
nan          0.863472
Name: emp_length_redefined, dtype: float64

The majority of the missing values in the job title (emp_title) supplied by the borrower coincides with the label n/a of employment length. This supports the idea that the 2 variables are related.

In [54]:
# let's do the same for those bororwers who reported
# the employer name

# number of borrowers for whom employer name is present
value = len(data.dropna(subset=['emp_title']))

# % of borrowers within each category
data.dropna(subset=['emp_title']).groupby(['emp_length_redefined'])['emp_length'].count().sort_values() / value

emp_length_redefined
nan          0.000000
10+ years    0.345476
0-10         0.654059
Name: emp_length, dtype: float64

The number of borrowers who have reported an employer name and indicate n/a as employment length are minimal. Further supporting the idea that the missing values in employment length and employment length are related.

'n/a' in 'employment length' could be supplied by people who are retired, or students, or self-employed. In all of those cases there would not be a number of years at employment to provide, therefore the customer would enter 'n/a' and leave empty the form at the side of 'employer_name'.

In a scenario like this, a missing value in the variable emp_title depends on or is related to the 'n/a' label in the variable emp_length. And, this missing value nature is, in principle, independent of the variable we want to predict (in this case whether the borrower will repay their loan). How this will affect the predictions is unknown.